In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


In [5]:
# !mkdir /content/VIBEdata/ 
%cd /content/VIBEdata/ 
!unzip -q /content/drive/My\ Drive/VIBE/CNN_Testing/250plus2000GandR.zip

/content/VIBEdata


In [ ]:
# !rm /content/KEYVIBEdata1/export/east/14281p2.jpg

In [ ]:
%cd '/content/drive/My Drive/VIBE/CNN_Testing/'
# !pip install -q pybullet
!pip install -q transform3d
!pip3 install -r requirements.txt
!pip3 install -q seaborn


In [1]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
device_name = torch.device('cuda:0')


1
Tesla K80


In [1]:
%cd '/content/drive/My Drive/VIBE/CNN_Testing/'
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import copy
import torch
import pandas as pd
import seaborn as sns
from torch import nn
from torch import optim
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from pathlib import Path
from PIL import Image
import os

data_dir = Path('/content/VIBEdata/250plus2000GandR') 

class CNN(nn.Module):
    def __init__(self, isnew = False, modelname = 'KEYVIBE_model.pth'):
        super(CNN, self).__init__()
        if isnew == True:
            self.model = models.resnet34(pretrained=True) #torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=True)
        else:
            self.model = torch.load(modelname)
            if torch.cuda.is_available():
              self.model.cuda()
        with open("action.txt", "r") as f:
            self.categories = [s.strip() for s in f.readlines()]
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
        # self.device = torch.device('cpu')
        self.train_losses, self.test_losses = [], []

    def load_split_train_test(self, data_dir, valid_size=.2):
        # train_transforms = transforms.Compose([transforms.Resize((256, 256)),
        #                                        transforms.ToTensor(),
        #                                        ])
        train_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        # test_transforms = transforms.Compose([transforms.Resize((256, 256)),
        #                                       transforms.ToTensor(),
        #                                       ])
        test_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        train_data = datasets.ImageFolder(data_dir, transform=train_transforms)
        test_data = datasets.ImageFolder(data_dir, transform=test_transforms)


        num_train = len(train_data)
        indices = list(range(num_train))
        split = int(np.floor(valid_size * num_train))
        np.random.shuffle(indices)

        from torch.utils.data.sampler import SubsetRandomSampler

        train_idx, test_idx = indices[split:], indices[:split]
        train_sampler = SubsetRandomSampler(train_idx)
        test_sampler = SubsetRandomSampler(test_idx)

        self.trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=64)
        self.testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=64)

    def train(self):
        # for param in self.model.parameters():
        #     param.requires_grad = False
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(nn.Linear(num_ftrs, 64),
                                      nn.ReLU(),
                                      nn.Dropout(.2),
                                      nn.Linear(64, 6),
                                      nn.LogSoftmax(dim=1))
        criterion = nn.NLLLoss().to(self.device)
        optimizer = optim.Adam(self.model.parameters(), lr=0.0005)
        # optimizer = optim.SGD(self.model.parameters(), lr=5e-4, momentum=0.9)
        for param in self.model.fc.parameters():
            param.requires_grad = False
        self.model.to(self.device)

        epochs = 20
        steps = 0
        running_loss = 0
        print_every = len(self.testloader)
        self.model.train()
        valid_losses = []
        Tflag = 1

        for epoch in range(epochs):
            for inputs, labels in self.trainloader:
                steps += 1
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                logps = self.model(inputs)
                loss = criterion(logps, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if steps % print_every == 0:
                    test_loss = 0
                    accuracy = 0
                    self.model.eval()
                    with torch.no_grad():
                        for inputs, labels in self.testloader:
                            inputs, labels = inputs.to(self.device),labels.to(self.device)
                            logps = self.model(inputs)
                            batch_loss = criterion(logps, labels)
                            test_loss += batch_loss.item()
                            ps = torch.exp(logps)
                            top_p, top_class = ps.topk(1, dim=1)
                            equals = top_class == labels.view(*top_class.shape)
                            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    self.train_losses.append(running_loss / len(self.testloader))
                    self.test_losses.append(test_loss / print_every)
                    print(f"Epoch {epoch + 1}/{epochs}.. "
                          f"Train loss: {running_loss / print_every:.3f}.. "
                          f"Test loss: {test_loss / print_every:.3f}.. "
                          f"Test accuracy: {accuracy / len(self.testloader):.3f}")
                    running_loss = 0
                    self.model.train()
                    valid_losses.append(test_loss)
                    if valid_losses[-1] == min(valid_losses):
                      self.best_model = copy.deepcopy(self.model)
                      torch.save(self.best_model, 'nathan_test.pth')
                    if  (Tflag == 1) & (epoch > 9): 
                      for param in self.model.parameters():
                          param.requires_grad = False
                      for param in self.model.fc.parameters():
                          param.requires_grad = True
                      Tflag = 0
                      print(Tflag)
                      optimizer = optim.Adam(self.model.fc.parameters(), lr=0.0001)
                      self.model.to(self.device)

            # if (self.test_losses[-1] < 0.1) & (self.train_losses[-1] <= 0.15):
            #   break
        if valid_losses[-1] == min(valid_losses):
                      self.best_model = copy.deepcopy(self.model)
                      torch.save(self.best_model, 'nathan_test.pth')

    def plot_loss(self):
        plt.plot(self.train_losses, label='Training loss')
        plt.plot(self.test_losses, label='Validation loss')
        plt.legend(frameon=False)
        plt.show()
      
    def plot_confusionmat(self):
        truelabels = []
        predictions = []
        self.model.eval()
        self.model.cpu()
        print("Getting predictions from test set...")
        for data, target in self.testloader:
            for label in target.data.numpy():
                truelabels.append(label)
            for prediction in self.model(data).data.numpy().argmax(1):
                predictions.append(prediction) 

        # Plot the confusion matrix
        cm = confusion_matrix(truelabels, predictions)
        tick_marks = np.arange(len(self.categories))

        df_cm = pd.DataFrame(cm, index = self.categories, columns = self.categories)
        plt.figure(figsize = (4,4))
        sns.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='g')
        plt.xlabel("Predicted Shape", fontsize = 20)
        plt.ylabel("True Shape", fontsize = 20)
        plt.show()


    def resize(self, path):
        # This was in case the resizing in the load function did not work
        dirs = os.listdir(path)
        for item in dirs:
            if os.path.isfile(path + item):
                im = Image.open(path + item)
                f, e = os.path.splitext(path + item)
                imResize = im.resize((200, 200), Image.ANTIALIAS)
                imResize.save(f + ' resized.jpg', 'JPEG', quality=90)

    def RUN(self, imgpath):
        self.model.eval()
        input_image = Image.open(imgpath)
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        # preprocess = transforms.Compose([transforms.Resize((256, 256)),
        #                                       transforms.ToTensor(),
        #                                       ])
        input_tensor = preprocess(input_image)
        input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model
        self.model.cpu()
        with torch.no_grad():
            output = self.model(input_batch)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        prob, catid = torch.topk(probabilities, 6)
        for i in range(prob.size(0)):
            print(self.categories[catid[i]], prob[i].item())
        print(probabilities)

# Resnet = CNN(True)
# Resnet.load_split_train_test(data_dir, .2)
# Resnet.train()
# Resnet.plot_loss()
# Resnet.plot_confusionmat()

/content/drive/MyDrive/University/SDU/VIBE/CNN_Testing


In [ ]:
  Resnet = CNN(True)
  Resnet.load_split_train_test(data_dir, .2)
  Resnet.train()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Epoch 1/20.. Train loss: 0.840.. Test loss: 0.660.. Test accuracy: 0.702
Epoch 1/20.. Train loss: 0.651.. Test loss: 0.634.. Test accuracy: 0.723
Epoch 1/20.. Train loss: 0.590.. Test loss: 0.600.. Test accuracy: 0.721
Epoch 2/20.. Train loss: 0.592.. Test loss: 0.509.. Test accuracy: 0.741
Epoch 2/20.. Train loss: 0.539.. Test loss: 0.539.. Test accuracy: 0.728
Epoch 2/20.. Train loss: 0.540.. Test loss: 0.614.. Test accuracy: 0.716
Epoch 2/20.. Train loss: 0.514.. Test loss: 0.472.. Test accuracy: 0.760
Epoch 3/20.. Train loss: 0.510.. Test loss: 0.488.. Test accuracy: 0.745
Epoch 3/20.. Train loss: 0.490.. Test loss: 0.473.. Test accuracy: 0.740
Epoch 3/20.. Train loss: 0.490.. Test loss: 0.453.. Test accuracy: 0.755
Epoch 3/20.. Train loss: 0.474.. Test loss: 0.489.. Test accuracy: 0.759
Epoch 4/20.. Train loss: 0.483.. Test loss: 0.453.. Test accuracy: 0.759
Epoch 4/20.. Train loss: 0.467.. Test loss: 0.460.. Test accuracy: 0.754
Epoch 4/20.. Train loss: 0.467.. Test loss: 0.497..

In [ ]:
  Resnet.plot_loss()
  Resnet.plot_confusionmat()

In [ ]:
Resnet = CNN(isnew=False, modelname='KEYVIBE_model_best_nathan.pth').cpu()
# Resnet.load_split_train_test(data_dir, .2)
# Resnet.plot_confusionmat()
Resnet.RUN('/content/KEYVIBEdata/export/west/1092p6.jpg')